In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

def stratified_patient_split(df, stratify_column='id', target_column='dose', n_bins=4, test_size=0.2, random_state=None):
    """
    Функция разбивает датафрейм на обучающий и тестовый наборы с учетом стратификации по медианным значениям целевой переменной каждого уникального элемента в столбце stratify_column.

    Параметры:
    - df (pd.DataFrame): исходный датафрейм
    - stratify_column (str): имя столбца, по уникальным значениям которого будет вычисляться медиана целевой переменной для стратификации
    - target_column (str): имя целевого столбца, по которому будет осуществляться стратификация
    - n_bins (int): количество интервалов для разделения медианных значений целевой переменной
    - test_size (float): доля данных, которая будет использоваться для тестового набора
    - random_state (int): seed для случайности при разбиении

    Возвращает:
    - train_df (pd.DataFrame): обучающий датафрейм
    - test_df (pd.DataFrame): тестовый датафрейм
    """
    # 1. Вычисляем медианное значение целевой переменной (target_column) для каждого пациента (stratify_column)
    # patient_medians - объект Series из pandas, где  индекс это ID пациента (значения из столбца stratify_column),
    # а значение — медианные значения дозы для каждого пациента из столбца target_column.
    patient_medians = df.groupby(stratify_column)[target_column].median()

    # 2. Определяем границы интервалов с учетом минимального и максимального значений целевой переменной (target_column)
    # отрезок между минимальной и максимальной дозой разбивается на равные интервалы в количестве n_bins. bins - массив с границами интервалов
    bins = np.linspace(df[target_column].min(), df[target_column].max(), n_bins + 1)

    # 3. Классифицируем медианные значения целевой переменной (target_column) по интервалам
    # binned_medians — объект Series, где: Индекс это ID пациента (stratify_column), а значение — это номер интервала, в который попало медианное значение для данного ID.
    binned_medians = pd.cut(patient_medians, bins, labels=False, include_lowest=True)

    # Особенность pd.cut - если количество интервалов большое, то в часть из них может не попасть ни одно значение или попасть только одно.
    # И то и другое не позволяет осуществлять стратификацию с помощью  train_test_split, поскольку в каждом интервале требуется не менее двух значений для обучающей и тестовоы выборок
    # Нижеследующий код до #4. направлен на поиск таких интервалов и присоединение их к соседним, имеющим больше одного значения

    # Рассчитываем количество пациентов в каждом интервале
    bin_counts = binned_medians.value_counts() #.value_counts() - это метод для объектов Series в pandas. Он подсчитывает, сколько раз каждое уникальное значение встречается в Series
    print("Пациентов в интервале:", bin_counts)

    # Если есть интервалы без пациентов. то присваиваем такому интервалу (с индексом i) значение 0
    for i in range(n_bins):
        if i not in bin_counts:
            bin_counts = pd.concat([bin_counts, pd.Series([0], index=[i])])

    # Находим проблемные интервалы, где меньше двух пациентов.
    problematic_bins = bin_counts[bin_counts < 2].index.tolist()
    print('Проблемные интервалы', problematic_bins)

    # Для каждого проблемного интервала пытаемся найти ближайший интервал с более чем одним пациентом
    for bin_num in problematic_bins:
        # Переменная для определения расстояния до проверяемого соседнего интервала
        dist = 1
        while True:
            lower_bin = bin_num - dist
            upper_bin = bin_num + dist
            # Если соседний интервал имеет более чем одного пациента, присваиваем проблемному интервалу номер соседнего интервала (объединяем интервалы)
            if lower_bin in bin_counts and bin_counts[lower_bin] > 1:
                binned_medians[binned_medians == bin_num] = lower_bin
                break
            elif upper_bin in bin_counts and bin_counts[upper_bin] > 1:
                binned_medians[binned_medians == bin_num] = upper_bin
                break
            # Увеличиваем расстояние, чтобы проверить следующие интервалы, если соседние не подошли по условиям
            dist += 1

    # Перерасчитываем количество пациентов в каждом интервале после переназначения проблемных интервалов
    bin_counts = binned_medians.value_counts()
    print("Пациентов в интервале после переназначения проблемных интервалов: ", bin_counts)

    # 4. Разделяем перечень уникальных ID пациентов на обучающую и тестовую выборки с учетом стратификации на основе интервалов по медианным значениям целевой переменной
    # в обучающую и тестовую выборки попадут пропорциональные значению test_size количества ID из каждого интервала binned_medians
    train_ids, test_ids = train_test_split(binned_medians.index, test_size=test_size, stratify=binned_medians, random_state=random_state)

    # 5. Разделяем исходный датафрейм на основе полученных выборок ID пациентов
    train_df = df[df[stratify_column].isin(train_ids)]
    test_df = df[df[stratify_column].isin(test_ids)]

    return train_df, test_df
